In [1]:
using Revise
using GeometryBasics: Vec3f, Point3f
using LinearAlgebra
using GLMakie
using StaticArrays
using VMRobotControl
using VMRobotControl.Splines: CubicSpline
using DifferentialEquations
using MeshIO
include("../functions.jl")

display_transform (generic function with 1 method)

## Importing ShadowHand URDF

### URDF Parsing

In [3]:
using FileIO, UUIDs
try
    FileIO.add_format(format"DAE", (), ".dae", [:DigitalAssetExchangeFormatIO => UUID("43182933-f65b-495a-9e05-4d939cea427d")])
catch
end

cfg = URDFParserConfig(;suppress_warnings=true) # This is just to hide warnings about unsupported URDF features
module_path = joinpath(splitpath(splitdir(pathof(VMRobotControl))[1])[1:end-1])
robot = parseURDF(joinpath(module_path, "URDFs/sr_description/sr_hand_vm_compatible.urdf"), cfg)

24DOF Mechanism{Float64} "shadowhand_motor" with 33 frames, 32 joints, 55 coordinates, 82 components

### Hand Visualization

In [8]:
# Compile robot model
m = compile(robot)
kcache = Observable(new_kinematics_cache(m))  # This stores the robot's joint state

# Create the figure
fig = Figure(size=(800, 600))

# Create interactive 3D scene
ls = LScene(fig[1, 1]; show_axis=true)  # 3D interactive scene

# Attach interactive camera controls
cam3d!(ls)  # Enables mouse interaction (rotate, zoom, pan)

# Plot the robot's visuals
robotvisualize!(ls, kcache)
display_frame(m, ls, "rh_ffmiddle")

# Display figure
display(fig)

GLMakie.Screen(...)

In [7]:
show(IOContext(stdout, :limit => false),  MIME("text/plain"), coordinates(vms))

Dict{String, VMRobotControl.CoordinateData} with 2 entries:
  "Target xy" => CoordSlice{2, String}(Target position, [1, 2])
  "Target position" => FramePoint{Float64, String}(.robot.world, [0.0, -0.1, 0.32])

## Creating the Virtual Mechanism System

### Gravity Compensation, Joint Limits and Joint Damping

In [9]:
# GRAVITY COMPENSATION

add_gravity_compensation!(robot, VMRobotControl.DEFAULT_GRAVITY)

# JOINT DAMPING and LIMIT SPRINGS

joint_limits = cfg.joint_limits

for joint_id in keys(joints(robot))
    limits = joint_limits[joint_id]
    isnothing(limits) && continue
    add_coordinate!(robot, JointSubspace(joint_id);  id="$(joint_id)_coord")
    @assert ~isnothing(limits.lower) && ~isnothing(limits.upper)
    add_deadzone_springs!(robot, 50.0, (limits.lower+0.1, limits.upper-0.1), "$(joint_id)_coord")
    add_component!(robot, LinearDamper(0.01, "$(joint_id)_coord"); id="$(joint_id)_damper")
end

### Virtual Mechanism Initialization

In [12]:
vms = VirtualMechanismSystem("myShadowVMS", robot)
root = root_frame(vms.robot)

# FINGER SPACING --> PRE-SHAPING
add_coordinate!(vms, CoordDifference(".robot.rh_ffmiddle_mass_coord", ".robot.rh_mfmiddle_mass_coord"); id="ff mf middle diff")
add_coordinate!(vms, CoordSlice("ff mf middle diff", SVector(1,3)); id = "ff mf middle diff xz")

add_coordinate!(vms, CoordNorm("ff mf middle diff xz"); id="ff mf middle planar norm")

fingers_spacing = 0.032
add_coordinate!(vms, ConstCoord(fingers_spacing); id = "fingers spacing")

add_coordinate!(vms, CoordDifference("ff mf middle planar norm", "fingers spacing"); id = "ff mf middle planar error")

K = 100.0
D = 50.0 

add_component!(vms, LinearSpring(K, "ff mf middle planar error");     id="ff mf middle spring")
add_component!(vms, LinearDamper(D, "ff mf middle planar error");     id="ff mf middle damper")

"ff mf middle damper"

## Simulating the Robot

### Setting Up the Simulation

In [16]:
using Logging: global_logger
using TerminalLoggers: TerminalLogger
global_logger(TerminalLogger())

disturbance_func(t) = mod(t, 6) < 3 ? SVector(0., 0., 0.) : SVector(0., 10., 0.)

f_setup(cache) = get_compiled_coordID(cache, ".robot.rh_fftip_mass_coord")

function f_control(cache, t, args, extra)
    tcp_pos_coord_id = args
    F = disturbance_func(t)
    uᵣ, uᵥ = get_u(cache)
    z = configuration(cache, tcp_pos_coord_id)
    J = jacobian(cache, tcp_pos_coord_id)
    mul!(uᵣ, J', F)
    nothing
end

tspan = (0., 10.)
vms_compiled = compile(vms)

q = (zero_q(vms_compiled.robot), zero_q(vms_compiled.virtual_mechanism)) # Robot joint angle, vm joint angles
q̇ = (zero_q̇(vms_compiled.robot), zero_q̇(vms_compiled.virtual_mechanism)) # Robot joint velocity, vm joint velocities

g = VMRobotControl.DEFAULT_GRAVITY
dcache = new_dynamics_cache(vms_compiled)
prob = get_ode_problem(dcache, g, q, q̇, tspan; f_setup, f_control)
@info "Simulating shadow robot with fingers spacing"

sol = solve(prob, Tsit5(), progress=true; maxiters=1e6, abstol=1e-6, reltol=1e-6);

[ Info: Simulating shadow robot with fingers spacing
ODE   0%|█                                              |  ETA: N/A
ODE   3%|██                                             |  ETA: 0:00:49
ODE   5%|███                                            |  ETA: 0:00:39
ODE   8%|████                                           |  ETA: 0:00:46
ODE  11%|██████                                         |  ETA: 0:00:41
ODE  14%|███████                                        |  ETA: 0:00:37
ODE  17%|█████████                                      |  ETA: 0:00:34
ODE  20%|██████████                                     |  ETA: 0:00:32
ODE  23%|███████████                                    |  ETA: 0:00:30
ODE  26%|█████████████                                  |  ETA: 0:00:28
ODE  29%|██████████████                                 |  ETA: 0:00:27
ODE  30%|███████████████                                |  ETA: 0:00:28
ODE  30%|███████████████                                |  ETA: 0:00:30
ODE  30%|██████

### Visualizing the Results

In [19]:
fig = Figure(; size=(720, 720), figure_padding=0)
display(fig)
ls = LScene(fig[1, 1]; show_axis=false)
cam = cam3d!(ls; center=false)
cam.lookat[] = [0.25, 0.5, 0.1]
cam.eyeposition[] = [-0.4, -0.8, 0.5]

plotting_t = Observable(0.0)
plotting_kcache = Observable(new_kinematics_cache(compile(vms)))
robotvisualize!(ls, plotting_kcache)

tcp_pos_id = get_compiled_coordID(plotting_kcache[], ".robot.rh_fftip_mass_coord")
tcp_pos = map(plotting_kcache) do kcache
    Point3f(configuration(kcache, tcp_pos_id))
end
force = map(t -> 0.01 * Vec3f(disturbance_func(t)), plotting_t)
arrowsize = map(f -> 0.1*(f'*f)^(0.25), force)
arrows!(ls, map(p -> [p], tcp_pos), map(f -> [f], force); color = :red, arrowsize)

fps = 60
T = sol.t[end]
N_frames = Int(floor(fps * T))
ts = LinRange(0.0, T, N_frames)
savepath = joinpath(module_path, "docs/src/assets/shadowhand_two_phases_ball_grasp.mp4")
display(fig)
animate_robot_odesolution(fig, sol, plotting_kcache, savepath; t=plotting_t);